In [2]:
import numpy as np
from scipy.integrate import quad

# Function to integrate
def f(s):
    return 1 / (1 + s**2)

# Composite Trapezoidal Rule
def trapezoidal(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return h * (0.5 * y[0] + np.sum(y[1:-1]) + 0.5 * y[-1])

# Composite Simpson's Rule
def simpsons(f, a, b, n):
    if n % 2 != 0:
        raise ValueError("n must be even")
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    return h / 3 * (y[0] + 4 * np.sum(y[1:n:2]) + 2 * np.sum(y[2:n-1:2]) + y[n])

# Exact value using analytical result
Ireal = 2 * np.arctan(5)

# Error estimation bounds
a, b = -5, 5

In [7]:
tol = 1e-4

def d2f(s):
    return np.abs((6 * s**2 - 2) / (1 + s**2)**3)

# find max of 2nd derivative
s_vals = np.linspace(a, b, 1000)
max_f2 = np.max(d2f(s_vals))

n_trap = int(np.ceil(np.sqrt(((b - a)**3 * max_f2) / (12 * tol))))

def d4f(s):
    return (384*s**4)/(1 + s**2)**5 - (288*s**2)/(1 + s**2)**4 + 24/(1 + s**2)**3

# f2_vals = d2f(s_vals)
# f4_approx = np.gradient(np.gradient(f2_vals, s_vals), s_vals)
# max_f4 = np.max(np.abs(f4_approx))
max_f4 = np.max(d4f(s_vals))

n_simp = int(np.ceil((((b - a)**5 * max_f4) / (180 * tol)) ** (1 / 4)))
if n_simp % 2 != 0:
    n_simp += 1  # Simpsons needs even n

# Apply rules
Tn = trapezoidal(f, a, b, n_trap)
Sn = simpsons(f, a, b, n_simp)

# Scipy integration for comparison
result_high_prec, _, info_high_prec = quad(f, a, b, epsabs=1e-6, full_output=1)
result_low_prec, _, info_low_prec = quad(f, a, b, epsabs=1e-4, full_output=1)

# Print results
print(f"Exact value: {Ireal:.15f}")
print(f"\nTrapezoidal:")
print(f"n = {n_trap}")
print(f"Approximation = {Tn}")
print(f"Error = {abs(Ireal - Tn)}")
print(f"Function evaluations = {n_trap + 1}")

print(f"\nSimpsons:")
print(f"n = {n_simp}")
print(f"Approximation = {Sn}")
print(f"Error = {abs(Ireal - Sn)}")
print(f"Function evaluations = {n_simp + 1}")

print(f"\nScipy quad:")
print(f"Tolerance 1e-6: Value = {result_high_prec}, Function calls = {info_high_prec['neval']}")
print(f"Tolerance 1e-4: Value = {result_low_prec}, Function calls = {info_low_prec['neval']}")

Exact value: 2.746801533890032

Trapezoidal:
n = 1291
Approximation = 2.7468013859623728
Error = 1.4792765901106009e-07
Function evaluations = 1292

Simpsons:
n = 108
Approximation = 2.746801528748203
Error = 5.14182874056246e-09
Function evaluations = 109

Scipy quad:
Tolerance 1e-6: Value = 2.7468015338900327, Function calls = 147
Tolerance 1e-4: Value = 2.746801533909586, Function calls = 63
